### Script Purpose
- Get BMF file 2014-16 from NCCS.
- Use only A confidence level records.
- Drop organizations that changed their NTEE codes between 14-16.

In [2]:
import pandas as pd
import requests
from tqdm import tqdm
import math
from irsx.xmlrunner import XMLRunner
xml_runner = XMLRunner()
import os

import warnings
warnings.filterwarnings('ignore')

In [3]:
df_bmf_1608=pd.read_csv('https://nccs-data.urban.org/data/bmf/2016/bmf.bm1608.csv')
df_bmf_1512=pd.read_csv('https://nccs-data.urban.org/data/bmf/2015/bmf.bm1512.csv')
df_bmf_1412=pd.read_csv('https://nccs-data.urban.org/data/bmf/2014/bmf.bm1412.csv')
df_bmf_1608['YR_MO']=1608
df_bmf_1512['YR_MO']=1512
df_bmf_1412['YR_MO']=1412

## Obtain `nteeConf==A`

In [3]:
df_bmf_14_16=pd.concat([df_bmf_1608, df_bmf_1412, df_bmf_1512], ignore_index=True)
df_bmf_14_16.loc[df_bmf_14_16[df_bmf_14_16.nteeConf=='a'].index, 'nteeConf']='A' # Correct erroneous records 'a'.
df_bmf_14_16_confA=df_bmf_14_16[df_bmf_14_16.nteeConf=='A'] # Select A confidence level records.

In [4]:
df_bmf_14_16.groupby('nteeConf').count()['EIN']/len(df_bmf_14_16)

nteeConf
A    0.561196
B    0.373232
C    0.065553
Name: EIN, dtype: float64

In [5]:
df_bmf_14_16_confA.sample(5)

,EIN,FIPS,NTEECC,FILER,ZFILER,NAME,ADDRESS,CITY,STATE,ZIP5,...,czFiler,cTaxPer,cAssets,cTotRev,cFinSrc,EPOST,FISYR_IMAGE,IRS990n,NAICS,YR_MO
3177192,133511640,36061.0,B90,Y,N,AMERICAN REGIONS MATHEMATICS LEAGUE INC,241 CENTRAL PARK WEST,NEW YORK,NY,10024,...,N,201412.0,147132.0,300506.0,bmf1512c3,Z,2014.0,0,611710.0,1512
3728044,364651294,54049.0,G25,N,N,STATE OF WEST VIRGINIA WEST VIRGINIA UNIVERSITY,586 MODS RUN RD,MANNINGTON,WV,26582,...,N,NaN,NaN,NaN,NaN,Z,NaN,0,813212.0,1512
2814054,742573896,48051.0,J40,Y,Y,TEXAS EXTENSION EDUCATION ASSOCIATION,PO BOX 125,SNOOK,TX,77878,...,Y,201312.0,NaN,NaN,submaster990n,Z,NaN,1,813930.0,1412
2434927,460518624,48157.0,P40,Y,Y,MOMS CLUB,2823 COTTON STOCK DR,SUGAR LAND,TX,77479,...,Y,201306.0,NaN,NaN,submaster990n,Z,NaN,1,624100.0,1412
3707820,363322126,46081.0,X21,N,N,ST JOHNS EPISCOPAL CHURCH ENDOWMENT TRUST,PO BOX 130,DEADWOOD,SD,57732,...,N,NaN,NaN,NaN,NaN,Z,NaN,0,813110.0,1512


**Drop organizations that changed their NTEE codes between 14-16.**

In [6]:
import ipyparallel as ipp
c = ipp.Client()
print(c.ids)
dview = c[:]

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39]


In [7]:
dview.execute('import pandas as pd')
dview['df_bmf_14_16_confA']=df_bmf_14_16_confA
dview['df_bmf_14_16_confA_ntee_chg']=pd.DataFrame()

In [8]:
# Orgs changed their NTEE codes.
@dview.parallel(block=True)
def ntee_chg(ein):
    global df_bmf_14_16_confA, df_bmf_14_16_confA_ntee_chg
    ntee1_list=df_bmf_14_16_confA[df_bmf_14_16_confA.EIN==ein]['NTEE1']
    if len(set(ntee1_list))!=1:
        df_bmf_14_16_confA_ntee_chg=pd.concat([df_bmf_14_16_confA_ntee_chg, 
                                               df_bmf_14_16_confA[df_bmf_14_16_confA.EIN==ein]]
                                             )

In [9]:
t=ntee_chg.map(df_bmf_14_16_confA.EIN.unique())

In [15]:
df_bmf_14_16_confA_ntee_chg=pd.concat(dview.gather('df_bmf_14_16_confA_ntee_chg'), ignore_index=False)
df_bmf_14_16_confA_ntee_chg[['EIN', 'YR_MO', 'NTEE1']].head(5)

,EIN,YR_MO,NTEE1
65,10063430,1608,C
1584662,10063430,1412,Y
3081824,10063430,1512,Y
113,10132625,1608,U
1584710,10132625,1412,Y


In [16]:
dview['ein_drop_list']=df_bmf_14_16_confA_ntee_chg.EIN.unique()
dview['df_bmf_14_16_confA_chg_drop']=pd.DataFrame()

In [17]:
@dview.parallel(block=True)
def func_ntee_chg_drop(ein):
    global df_bmf_14_16_confA_chg_drop, df_bmf_14_16_confA
    if ein not in ein_drop_list:
        df_bmf_14_16_confA_chg_drop=pd.concat([df_bmf_14_16_confA_chg_drop, 
                                               df_bmf_14_16_confA[df_bmf_14_16_confA.EIN==ein]
                                              ])

In [18]:
t=func_ntee_chg_drop.map(df_bmf_14_16_confA.EIN.unique())

In [19]:
df_bmf_14_16_confA_chg_drop=pd.concat(dview.gather('df_bmf_14_16_confA_chg_drop')).sort_index()

In [24]:
len(df_bmf_14_16_confA_chg_drop)

2548479

In [23]:
df_to_write=df_bmf_14_16_confA_chg_drop
num_file=4
file_path_name='../../dataset/df_bmf_14_16_confA_chg_drop.pkl.gz/df_bmf_14_16_confA_chg_drop.pkl.gz'
for index in range(0, df_to_write.iloc[-1].name+1, math.ceil(df_to_write.iloc[-1].name/num_file)):
    df_temp=df_to_write.loc[index:index+math.ceil(df_to_write.iloc[-1].name/num_file)-1]
    df_temp.to_pickle(file_path_name+'-'+str(df_temp.iloc[0].name)+'-'+str(df_temp.iloc[-1].name), compression='gzip')

In [26]:
# Test reading file.
import os
file_list=os.listdir('../../dataset/df_bmf_14_16_confA_chg_drop.pkl.gz/')
df_test=pd.DataFrame()
for file in file_list:
    df_test=pd.concat([df_test, 
                       pd.read_pickle('../../dataset/df_bmf_14_16_confA_chg_drop.pkl.gz/'+file, compression='gzip')])
len(df_test)

2548479

**Done with data acquisition, see how many changed.**

In [20]:
df_bmf_14_16_confA_chg_drop=pd.DataFrame()
for file in os.listdir('../../dataset/df_bmf_14_16_confA_chg_drop.pkl.gz/'):
    df_bmf_14_16_confA_chg_drop=pd.concat([df_bmf_14_16_confA_chg_drop,
                                           pd.read_pickle('../../dataset/df_bmf_14_16_confA_chg_drop.pkl.gz/'+file, compression='gzip')
                                          ])

In [23]:
[len(df_bmf_14_16_confA_chg_drop.EIN.unique()), 
 len(df_bmf_14_16_confA), 
 len(df_bmf_14_16_confA_chg_drop.EIN.unique())/len(df_bmf_14_16_confA)
 1-len(df_bmf_14_16_confA_chg_drop.EIN.unique())/len(df_bmf_14_16_confA)
]

911423

In [44]:
len(df_bmf_14_16_confA.EIN.unique())
# ~1.76% Changed their NTEE codes between 2014-2016. Drop these records since we have no idea on exactly when these codes were changed.

(101045, 960315)

In [45]:
len(df_bmf_14_16_confA[['EIN', 'NTEE1']].drop_duplicates()), len(df_bmf_14_16_confA.EIN.unique())
# ~1.76% Changed their NTEE codes between 2014-2016. Drop these records since we have no idea on exactly when these codes were changed.

(977566, 960315)

In [40]:
df_bmf_14_16_confA_rm_chg=df_bmf_14_16_confA.loc[df_bmf_14_16_confA[['EIN', 'NTEE1']].drop_duplicates(keep=False).index]

In [43]:
len(df_bmf_14_16_confA_rm_chg[['EIN', 'NTEE1']].drop_duplicates(keep=False)), len(df_bmf_14_16_confA_rm_chg)

(199801, 214947)

In [14]:
len(df_bmf_14_16[['EIN', 'NTEE1', 'nteeConf']].drop_duplicates()), len(df_bmf_14_16[['EIN', 'NTEE1', 'nteeConf', 'YR_MO']].drop_duplicates())

(1792488, 4633436)

## Obtain a `nteeConf` random sample

In [13]:
df_bmf_14_16=pd.concat([df_bmf_1608, df_bmf_1412, df_bmf_1512], ignore_index=True)
df_bmf_14_16_sample20pt=df_bmf_14_16.sample(round(len(df_bmf_14_16)*.2))
df_bmf_14_16_sample20pt

,EIN,FIPS,NTEECC,FILER,ZFILER,NAME,ADDRESS,CITY,STATE,ZIP5,...,czFiler,cTaxPer,cAssets,cTotRev,cFinSrc,EPOST,FISYR_IMAGE,IRS990n,NAICS,YR_MO
2259989,376089889,17163.0,D32,N,N,MISSISSIPPI VALLEY DUCK HUNTERS ASSOCIATION OF...,PO BOX 166,BELLEVILLE,IL,62222,...,N,NaN,NaN,NaN,NaN,Z,2014.0,0,712190.0,1412
54256,60646633,9009.0,P28,Y,N,FARNAM-NEIGHBORHOOD HOUSE INC,162 FILLMORE ST,NEW HAVEN,CT,6513,...,N,201506.0,1725024.0,776466.0,bmf1608c3,Z,2014.0,0,624190.0,1608
4574090,941606518,6041.0,P82,Y,N,CEDARS OF MARIN,PO BOX 947,ROSS,CA,94957,...,N,201409.0,16173727.0,7842668.0,bmf1512c3,Z,2014.0,0,624120.0,1512
2812025,742484671,4013.0,T20,Y,N,PARACLETE FOUNDATION INC,6125 E KELTON LN,SCOTTSDALE,AZ,85254,...,N,201212.0,543628.0,12231.0,core2012pf,Z,2013.0,0,813211.0,1412
3602623,310973514,18103.0,B112,N,N,MACONAQUAH BAND BOOSTERS INC,PO BOX 73,BUNKER HILL,IN,46914,...,N,201305.0,12360.0,-2448.0,core2013pc,Z,2013.0,0,813219.0,1512
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1974406,261385776,48167.0,T22,Y,N,SASSER FAMILY FOUNDATION INC,7221 BROADWAY ST,GALVESTON,TX,77554,...,N,201212.0,625871.0,37486.0,core2012pf,Z,2013.0,0,813211.0,1412
3798249,391866588,55141.0,N61,Y,N,UNDERWIRE HOLDINGS INC,PO BOX 8000,WISC RAPIDS,WI,54495,...,N,201312.0,131517.0,74137.0,core2013co,Z,2014.0,0,713990.0,1512
2490587,464707100,51059.0,X99,N,N,CAOL AIT,13315 TWIN LAKES DR,CLIFTON,VA,20124,...,N,NaN,NaN,NaN,NaN,Z,NaN,0,813110.0,1412
4532139,911233120,53067.0,N60,Y,N,PUGET SOUND ENDURO RIDERS,5947 HIDDEN MEADOWS LN SW,OLYMPIA,WA,98512,...,N,201412.0,82823.0,60221.0,bmf1512c3,Z,2014.0,0,713990.0,1512


In [14]:
df_bmf_14_16_sample20pt.to_pickle('../../dataset/intermediary/df_bmf_14_16_sample20pt.pkl.bz2', compression='bz2')

## `nteeConf==B` sample

In [16]:
df_bmf_14_16=pd.concat([df_bmf_1608, df_bmf_1412, df_bmf_1512], ignore_index=True)
df_bmf_14_16_nteeConfB=df_bmf_14_16[df_bmf_14_16.nteeConf=='B']
df_bmf_14_16_nteeConfB_sample20pt=df_bmf_14_16_nteeConfB.sample(round(len(df_bmf_14_16_nteeConfB)*.2))
df_bmf_14_16_nteeConfB_sample20pt

,EIN,FIPS,NTEECC,FILER,ZFILER,NAME,ADDRESS,CITY,STATE,ZIP5,...,czFiler,cTaxPer,cAssets,cTotRev,cFinSrc,EPOST,FISYR_IMAGE,IRS990n,NAICS,YR_MO
849231,461102565,6071.0,W30,Y,N,THANKS 4 SERVING,19095 MUNSEE RD,APPLE VALLEY,CA,92307,...,N,201512.0,1069.0,1074.0,bmf1608c3,Z,2014.0,0,813410.0,1608
3049521,951655614,6071.0,P40,Y,N,FAMILY SERVICE ASSOCIATION OF REDLANDS,612 LAWTON ST,REDLANDS,CA,92374,...,N,201312.0,4206031.0,1088917.0,bmf1412c3,Z,2013.0,0,624100.0,1412
3376835,237160188,42075.0,B99,Y,N,AMERICAN ASSOCIATION OF CHAIRS OF DEPARTMENT O...,1301 WOODLAND ST,LEBANON,PA,17042,...,N,201406.0,202122.0,10403.0,bmf1512c3,Z,2015.0,0,611000.0,1512
3667548,341957292,39037.0,B11,Y,N,GREENVILLE OHIO SCHOOLS FOUNDATION INC,PO BOX 1073,GREENVILLE,OH,45331,...,N,201412.0,503536.0,36929.0,bmf1512c3,Z,2014.0,0,813219.0,1512
2216492,363368854,17031.0,L40,Y,N,BOBBY E WRIGHT HOUSING COMPLEX INC,566 WEST LAKE STREET,CHICAGO,IL,60661,...,N,201306.0,1314686.0,693575.0,core2012pc,Z,2014.0,0,721199.0,1412
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2196904,352200461,17091.0,S30,Y,N,GRUNDY LIVINGSTON KANKAKEE WORKFORCE INVESTMEN...,187 S SCHUYLER AVE STE 560,KANKAKEE,IL,60901,...,N,201306.0,18591.0,202106.0,bmf1412c3,Z,2014.0,0,813410.0,1412
4238252,592881813,12063.0,X20,N,N,NEW HOPE ASSEMBLY OF GOD,1314 BUNKER RD,GRACEVILLE,FL,32440,...,N,NaN,NaN,NaN,NaN,Z,NaN,0,813110.0,1512
1262186,710865765,48187.0,I40,Y,Y,GUADALUPE COUNTY DETENTION MINISTRIES INC,PO BOX 1402,SEGUIN,TX,78156,...,Y,201412.0,NaN,NaN,submaster990n,Z,NaN,1,541199.0,1608
3343363,232951382,42091.0,N63,Y,N,RICHIE ASHBURN FOUNDATION,PO BOX 1564,BLUE BELL,PA,19422,...,N,201412.0,44430.0,73502.0,bmf1512c3,Z,2014.0,0,713990.0,1512


In [17]:
df_bmf_14_16_nteeConfB_sample20pt.to_pickle('../../dataset/intermediary/df_bmf_14_16_nteeConfB_sample20pt.pkl.bz2', compression='bz2')

## `nteeConf==C` sample

In [18]:
df_bmf_14_16=pd.concat([df_bmf_1608, df_bmf_1412, df_bmf_1512], ignore_index=True)
df_bmf_14_16_nteeConfC=df_bmf_14_16[df_bmf_14_16.nteeConf=='C']
df_bmf_14_16_nteeConfC_sample20pt=df_bmf_14_16_nteeConfC.sample(round(len(df_bmf_14_16_nteeConfC)*.2))
df_bmf_14_16_nteeConfC_sample20pt

,EIN,FIPS,NTEECC,FILER,ZFILER,NAME,ADDRESS,CITY,STATE,ZIP5,...,czFiler,cTaxPer,cAssets,cTotRev,cFinSrc,EPOST,FISYR_IMAGE,IRS990n,NAICS,YR_MO
1672566,116313080,36085.0,T20,Y,N,CHESED AVRHOM HACOHN FOUNDATION,201 EDWARD CURRY AVENUE,STATEN ISLAND,NY,10314,...,N,201212.0,219484.0,7765.0,core2012pf,Z,2013.0,0,813211.0,1412
3046484,946121463,6041.0,S99,Y,N,WESTERN CANDY CONFERENCE,1500 GRANT AVE STE 200,NOVATO,CA,94945,...,N,201405.0,78157.0,135043.0,bmf1412co,Z,2014.0,0,813410.0,1412
1565797,954111973,17043.0,E99,Y,Y,COMPASSIONATE FRIENDS INC,PO BOX 3696,OAK BROOK,IL,60522,...,Y,201512.0,NaN,NaN,submaster990n,Z,NaN,1,621999.0,1608
2299482,391501410,55071.0,S02,Y,Y,INSTITUTE OF MANAGEMENT ACCOUNTANTS INC,5019 ELMER LN,MANITOWOC,WI,54220,...,Y,201406.0,NaN,NaN,submaster990n,Z,NaN,1,541618.0,1412
2698326,596057118,12086.0,B99,N,N,CONCORDIA LUTHERAN SCHOOL,8701 SW 124TH ST,MIAMI,FL,33176,...,N,NaN,NaN,NaN,NaN,Z,NaN,0,611000.0,1412
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
532867,311651365,18097.0,S99,N,N,ZIONSVILLE MUNICIPAL BUILDING CORPORATION,1 INDIANA SQ STE 2800,INDIANAPOLIS,IN,46204,...,N,NaN,NaN,NaN,NaN,Z,NaN,0,813410.0,1608
2680773,592113581,12109.0,T20,N,N,ST PHOTIOS FOUNDATION INC,41 SAINT GEORGE ST,ST AUGUSTINE,FL,32084,...,N,NaN,NaN,NaN,NaN,Z,NaN,0,813211.0,1412
1559515,953333758,39149.0,N50,Y,Y,WALLY BYAM CARAVAN CLUB INTERNATIONAL INC,803 E PIKE ST,JACKSON CTR,OH,45334,...,Y,201509.0,NaN,NaN,submaster990n,Z,NaN,1,713990.0,1608
2703074,610957218,21089.0,A80,Y,Y,DAUGHTERS OF AMERICA,119 SECOND ST,SOUTH SHORE,KY,41175,...,Y,201312.0,NaN,NaN,submaster990n,Z,NaN,1,712120.0,1412


In [19]:
df_bmf_14_16_nteeConfC_sample20pt.to_pickle('../../dataset/intermediary/df_bmf_14_16_nteeConfC_sample20pt.pkl.bz2', compression='bz2')